# VacationPy

In [1]:
# import libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import google_apikey

In [2]:
#getting the csv file created in weatherpy
city_weather_data = pd.read_csv("../WeatherPy/output_data/formatted_cities.csv")

city_weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Qui Nhon,13.7667,109.2333,80.17,75,78,5.32,VN,1618531899
1,Mkushi,-13.6202,29.3939,57.70,83,96,4.65,ZM,1618531906
2,Avarua,-21.2078,-159.7750,84.20,65,14,10.36,CK,1618531913
3,Faanui,-16.4833,-151.7500,82.63,69,33,10.54,PF,1618531919
4,Bluff,-46.6000,168.3333,55.99,69,74,13.00,NZ,1618531926


# Humidity Heatmap

In [3]:
#configure gmaps
gmaps.configure(api_key=google_apikey)

#lists to hold data
locations = []
humidities = []

#pull locations and humidities out of the data frame (Hasanov, n.d.)
for index, city in city_weather_data.iterrows():
    #create a tuple with the latitude and longitude values and appending to the location list
    loc = (city["Lat"], city["Lng"])
    locations.append(loc)
    #appending the humidity value to the humidities list
    humidities.append(city["Humidity"])

In [4]:
#customizing the figure
figure_layout = {
    'width': '1100px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
#create the figure
fig = gmaps.figure(layout=figure_layout)

#create the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidities)

#add the heat layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…

# Vacation Criteria Data Frame

In [5]:
#set lists for indicies
bad_vacation_index = []

#go through ideal conditions and pull any non-compatible indicies
for index, city in city_weather_data.iterrows():
    #find a good max_temperature
    if city["Max Temp"] > 80:
        if index not in bad_vacation_index:
            bad_vacation_index.append(index)
    elif city["Max Temp"] < 70:
        if index not in bad_vacation_index:
            bad_vacation_index.append(index)
    #find low wind speeds
    elif city["Wind Speed"] > 10:
        if index not in bad_vacation_index:
            bad_vacation_index.append(index)
    #find comfortable cloud coverage
    elif city["Cloudiness"] < 20:
        if index not in bad_vacation_index:
            bad_vacation_index.append(index)
    #find comfortable humidity
    elif city["Humidity"] > 50:
        if index not in bad_vacation_index:
            bad_vacation_index.append(index)

#drop the non-ideal vacation locations
ideal_vacation_cities = city_weather_data.drop(index=bad_vacation_index)

print(len(ideal_vacation_cities))
ideal_vacation_cities

8


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
115,Vredendal,-31.6683,18.5012,76.21,42,58,9.19,ZA,1618532770
122,Lüderitz,-26.6481,15.1594,77.72,28,78,4.21,NaN,1618532816
162,Tezu,27.9167,96.1667,79.95,34,100,0.60,IN,1618533087
286,Jos,9.9167,8.9000,78.04,24,100,1.48,NG,1618533916
378,Port Macquarie,-31.4333,152.9167,73.99,42,76,3.00,AU,1618534388
475,Nizwá,22.9333,57.5333,75.81,26,27,2.06,OM,1618535203
489,Kang,-23.6752,22.7876,70.66,33,98,7.58,BW,1618535300
496,Tura,25.5198,90.2201,79.97,45,22,5.32,IN,1618535351


# Hotel Map

In [16]:
#create new column in dataframe
ideal_vacation_cities["Hotel Name"] = ""

#creating function for finding the first hotel and adding it to the dataframe
for index, city in ideal_vacation_cities.iterrows():

    #set up search parameters
    target_coordinates = f"{str(city[1])},{str(city[2])}"
    target_radius = 5000
    target_type = "lodging"

    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": google_apikey,
    }

    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #run request for the hotel search
    response = requests.get(base_url, params=params)

    # convert response to json
    hotel_data = response.json()

    #put the hotel name into the dataframe
    ideal_vacation_cities.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]

ideal_vacation_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
115,Vredendal,-31.6683,18.5012,76.21,42,58,9.19,ZA,1618532770,Tharrakamma Guest House
122,Lüderitz,-26.6481,15.1594,77.72,28,78,4.21,NaN,1618532816,LÜDERITZ NEST HOTEL
162,Tezu,27.9167,96.1667,79.95,34,100,0.60,IN,1618533087,Circuit House
286,Jos,9.9167,8.9000,78.04,24,100,1.48,NG,1618533916,Silk Suites
378,Port Macquarie,-31.4333,152.9167,73.99,42,76,3.00,AU,1618534388,Rydges Port Macquarie
475,Nizwá,22.9333,57.5333,75.81,26,27,2.06,OM,1618535203,Orient Apartments
489,Kang,-23.6752,22.7876,70.66,33,98,7.58,BW,1618535300,Kang Ultra Stop
496,Tura,25.5198,90.2201,79.97,45,22,5.32,IN,1618535351,Hotel RIKMAN Continental


In [21]:
#code for the marker boxes
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in ideal_vacation_cities.iterrows()]
locations = ideal_vacation_cities[["Lat", "Lng"]]

#creating the marker layer
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
#adding the marker layer to the heatmap from earlier
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…